# `System.Text.Json.Nodes`

>Provides types for handling an in-memory writeable document object model (DOM) for random access of the JSON elements within a structured view of the data. [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.nodes)]

Based on my experience with the older class definitions under `System.Text.Json`, I interpret the quote above as a relief to the the frustrating limitations of having a high-performance read-only API for JSON. However, there is a warning in the phrase “in-memory writeable document,” starting with _in-memory_: surely, anything done with the class definitions under `System.Text.Json.Nodes` will be resource expensive.

Also, it may help to summarize the .NET 6 caveats for `System.Text.Json.Nodes`:

- there is no `JsonNode.GetValueKind` method (do not let the Microsoft docs fool you 🧐)
- there is no `JsonNode.DeepClone` method (more on this below)

## my weird extension method, `AsJupyterDisplayDictionary` 😐

The following extension method `AsJupyterDisplayDictionary` is needed here because displaying `JsonNode` in a .NET Interactive Jupyter Notebook is effectively infinitely recursive (making display on GitHub problematic):

In [1]:
using System.Text.Json;
using System.Text.Json.Nodes;

public static Dictionary<string, string> AsJupyterDisplayDictionary(this JsonNode node)
{
    return node.GetValueKind() switch
    {
        JsonValueKind.Object =>
            node
                .AsObject()
                .ToDictionary(i => i.Key, j => j.Value.ToJsonString(new JsonSerializerOptions { WriteIndented = true })),
        _ => throw new InvalidOperationException ($"The expected {nameof(JsonObject)} is not here.")
    };
}

## start with `JsonNode`

To “get started” with `System.Text.Json.Nodes`, I assume we start with `JsonNode.Parse` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.parse)] :

In [2]:
var jsonString = @"
    {
        ""top"": {
            ""one"": ""this is first"",
            ""two"": true,
            ""two-point-five"": 2.5,
            ""three"": {
                ""p1"": ""this is three-point-one"",
                ""p2"": ""this is three-point-two"",
                ""my-date"": ""2019-11-22T05:58:34.573Z""
            }
        }
    }
";

var jNode = JsonNode.Parse(jsonString);

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5, ""three"": { ""p1"": ""this is three-point-one"", ""p2"": ""this is three-point-two"", ""my-date"": ""2019-11-22T05:58:34.573Z"" } }"


## the `JsonNode` indexer

The most direct and terse way to traverse our `jNode` is to use indexers (`[]`). For example, we can examine the types of the children of our `jNode`:

In [3]:
jNode["top"].GetType()

System.Text.Json.Nodes.JsonObject

In [4]:
jNode["top"]["one"].GetType()

System.Text.Json.Nodes.JsonValueOfElement

In [5]:
jNode["top"]["three"].GetType()

System.Text.Json.Nodes.JsonObject

There are two things I must mention while looking at the output above:

1. `JsonNode` contains `JsonElement` and `JsonObject` instead of child `JsonNode`s.
2. Both `JsonObject` and `JsonArray` inherit from `JsonNode`.

## when indexer traversal goes wrong

The problem with indexers arises when the traversal is not always along “the happy path”:

In [6]:
jNode["foo"]["one"]

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#6.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Exceptions can be avoided with the null-conditional operator:

In [7]:
jNode["foo"]?["one"]

<null>

When we need to avoid propagating `null`, we can perform document traversal with `TryGetPropertyValue` (see below). Also, for those F# lovers out there, I have [my most advanced way to traverse](https://github.com/BryanWilhite/Songhay.Modules/blob/main/Songhay.Modules.Tests/JsonDocumentUtilityTests.fs) `JsonDocument`.

## `JsonElement` and `JsonNode`

Document traversal with `JsonElement` is done with its `Get*` and `TryGet*` methods. In the world of `JsonNode`, we can depend on `JsonObject` (via the `AsObject` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.asobject?view=net-8.0)]) and then call `TryGetPropertyValue`:

In [8]:
jNode
    .AsObject()
    .TryGetPropertyValue("top", out JsonNode topNode);

topNode
    .AsObject()
    .TryGetPropertyValue("one", out JsonNode oneNode);

oneNode.GetValue<string>()

this is first

This `TryGetPropertyValue` approach seems a bit clunky. Had I not stumbled upon “[.NET 6: Modify JSON in Memory with the System.Text.Json.Nodes Namespace](https://deliverystack.net/2021/12/06/net-6-modify-json-in-memory-with-the-system-text-json-nodes-namespace/)” by John West I would have gone to bed for a few nights not knowing that Microsoft wants us to use a LINQish approach to traversal:

In [9]:
JsonNode oneNode = jNode
    .AsObject()
    .FirstOrDefault(i => i.Key == "top")
    .Value?
    .AsObject()
    .FirstOrDefault(i => i.Key == "one")
    .Value;

oneNode?.GetValue<string>()

this is first

This _fluent_ approach allows us to handle not-so-happy JSON document traversal paths:

In [10]:
jNode
    .AsObject()
    .FirstOrDefault(i => i.Key == "foo")
    .Value?
    .AsObject()
    .FirstOrDefault(i => i.Key == "one")
    .Value?
    .GetValue<string>()

<null>

There is a very important subtlety in the design of `JsonObject` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonobject?view=net-8.0)]: `JsonObject` implements several collection interfaces, including `ICollection<KeyValuePair<String,JsonNode>>`. This is why we can call LINQ’s `FirstOrDefault` directly from an `AsObject()` invocation.

## the forward-only design of `JsonElement` in contrast to `JsonNode`

Earlier, we obtained `oneNode` by `TryGet*`ing and via the LINQish approach. Both of the outputs above are of type `JsonElement`:

In [11]:
oneNode.GetType()

System.Text.Json.Nodes.JsonValueOfElement

We can obtain `JsonElement` from `JsonNode` with:

In [12]:
using System.Text.Json;

var oneElement = oneNode.GetValue<JsonElement>();

oneElement

"this is first"

In [13]:
oneElement.GetType()

System.Text.Json.JsonElement

The sadness here is knowing that once we have `oneElement` there is no way to go back to `oneNode` and have all of that memory-expensive `Parent` and `Root` information 😦 We can see this loss by looking at this:

In [14]:
oneElement.GetRawText()

"this is first"

For the sake of read-only, forward-only efficiency, `JsonElement` only knows about its children (not even about itself 😲).

## the `JsonNode.GetValue<T>` method

As seen above, call `JsonNode.GetValue<T>` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.getvalue?view=net-6.0)] to obtain `JsonNode` values:

In [15]:
jNode["top"]?["one"]?.GetValue<string>()

this is first

In [16]:
jNode["top"]?["two"]?.GetValue<bool>()

True

In [17]:
jNode["top"]?["two-point-five"]?.GetValue<decimal>()

2.5

## the importance of the `JsonNode.Parent` property

The `JsonNode.Parent` property [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.parent)] is why we are in this namespace in the first place. With some expense, we are freed from the forward-only design goals of `JsonElement`. We see this property (along with the `JsonNode.Root` property [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.root)]) in the output above.

We can use the `JsonNode.Parent` property to discover the name of our `oneNode` element, starting with the following LINQ projection:

In [18]:
oneNode.Parent.AsObject().Select(i => i.Key).ToArray()

[ one, two, two-point-five, three ]

We can see clearly that we have been leveraging `KeyValuePair<string, JsonNode>.Key` to list all of the keys (property names) under our `top` root element. We may be correct when we put our trust in `JsonNode` equality when we state the following:

In [19]:
KeyValuePair<string, JsonNode> pair = oneNode
    .Parent
    .AsObject()
    .FirstOrDefault(i => i.Value == oneNode);

string.Format($"({pair.Key}, {pair.Value.GetValue<string>()})")

(one, this is first)

Are we correct when we put our trust in `JsonNode` equality? We can be more explicit to avoid getting into the weeds:

In [20]:
pair = oneNode
    .Parent
    .AsObject()
    .Where(i => i.Value.GetValueKind() == JsonValueKind.String)
    .FirstOrDefault(i => i.Value.GetValue<string>().ToLowerInvariant() == oneNode.GetValue<string>()
    .ToLowerInvariant());

string.Format($"({pair.Key}, {pair.Value.GetValue<string>()})")

(one, this is first)

## `JsonObject` and `IEnumerable<KeyValuePair<String,JsonNode>>`

`JsonObject` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonobject)] looks like the Microsoft-equivalent of the Newtonsoft `JObject` [📖 [docs](https://www.newtonsoft.com/json/help/html/T_Newtonsoft_Json_Linq_JObject.htm)]. One major difference between `JsonObject` and `JObject` is the the capability `JsonObject` has to construct from `ICollection<KeyValuePair<String,JsonNode>>`:

In [21]:
var dictionary = new Dictionary<string, JsonNode>
{
    { "one", "uno" },
    { "two", 2 },
    { "isThree", false },
};

var jO = new JsonObject(dictionary);

var propertyName = "four";

jO.Add(propertyName, "quarto");

jO.AsJupyterDisplayDictionary()

key,value
one,"""uno"""
two,2
isThree,false
four,"""quarto"""


In [22]:
oneNode
    .Parent
    .AsJupyterDisplayDictionary()

key,value
one,"""this is first"""
two,true
two-point-five,2.5
three,"{ ""p1"": ""this is three-point-one"", ""p2"": ""this is three-point-two"", ""my-date"": ""2019-11-22T05:58:34.573Z"" }"


## what about `GetValue<DateTime>`?

Above we saw `GetValue<T>` working with “any primitive value supported by current `JsonElement`.” Is `DateTime` supported?

In [23]:
var dateTime = jNode["top"]["three"]["my-date"].GetValue<DateTime>();

$"year: {dateTime.Year}; hour: {dateTime.Hour}"

year: 2019; hour: 5

## actually deleting a child `JsonObject`

In our work here, recall that `top.three` is our child `JsonObject`:

In [24]:
jNode["top"]["three"].GetValueKind() == JsonValueKind.Object

True

The `JsonObject.Remove` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonobject.remove?view=net-8.0)] removes by property name. In this case, we have to traverse up to `top`:

In [25]:
jNode["top"].AsObject().Remove("three")

True

In [26]:
jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5 }"


## adding a child `JsonArray`

We can add an array to the `JsonObject`, `jNode["top"]`, by appending a new key `threeArray` and setting it in place:

In [27]:
jNode["top"]["threeArray"] = JsonArray.Parse("[]");

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5, ""threeArray"": [] }"


In [28]:
JsonArray a = jNode["top"]?["threeArray"].AsArray();

a.Add(JsonValue.Parse("{ \"id\": 1, \"value\": \"one\"}"));
a.Add(JsonValue.Parse("{ \"id\": 2, \"value\": \"two\"}"));
a.Add(JsonValue.Parse("{ \"id\": 3, \"value\": \"three\"}"));

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5, ""threeArray"": [ { ""id"": 1, ""value"": ""one"" }, { ""id"": 2, ""value"": ""two"" }, { ""id"": 3, ""value"": ""three"" } ] }"


## removing an item from a `JsonArray`

Before we see how an item in an array can be removed, let us see the effect of `JsonNode.GetPath` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.getpath?view=net-8.0)]:

In [29]:
jNode["top"]?["threeArray"]?[1]?.GetPath()

$.top.threeArray[1]

This is [JSON Path](https://en.wikipedia.org/wiki/JSONPath) syntax, indicating the _second_ item in our `0`-based array.

Now, let us try to remove this item using `JsonArray.Remove` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonarray.remove?view=net-8.0)]:

In [30]:
var child = jNode["top"]?["threeArray"]
    .AsArray()
    .OfType<JsonObject>()
    .First(jO => jO["id"].GetValue<int>() == 2);

jNode["top"]?["threeArray"]
    .AsArray()
    .Remove(child);

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5, ""threeArray"": [ { ""id"": 1, ""value"": ""one"" }, { ""id"": 3, ""value"": ""three"" } ] }"


`JsonArray.Remove` deletes _by value_ instead of by array index (see `JsonArray.RemoveAt` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonarray.removeat?view=net-8.0)]).

## you cannot rename a `JsonObject` property name…

…you can only replace it with a new property, having the same value:

In [31]:
JsonValue jV = jNode["top"]?["one"]?.AsValue();

jNode["top"]["one-changed"] = jV.GetValue<string>();
jNode["top"].AsObject().Remove("one");

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""two"": true, ""two-point-five"": 2.5, ""threeArray"": [ { ""id"": 1, ""value"": ""one"" }, { ""id"": 3, ""value"": ""three"" } ], ""one-changed"": ""this is first"" }"


## calling `JsonNode.GetValue<JsonObject>` will _always_ throw an exception

Let us go back to the original value of `jNode`:

In [32]:
jNode = JsonNode.Parse(jsonString);

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5, ""three"": { ""p1"": ""this is three-point-one"", ""p2"": ""this is three-point-two"", ""my-date"": ""2019-11-22T05:58:34.573Z"" } }"


Our last call of `JsonNode.GetValue<string>` (above) might encourage us to do something like this:

In [33]:
jNode["top"]?["three"]?.GetValue<JsonObject>()

Error: System.InvalidOperationException: The node must be of type 'JsonValue'.
   at System.Text.Json.Nodes.JsonNode.GetValue[T]()
   at Submission#33.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Microsoft (in this current time-frame) is [telling us](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.getvalue?view=net-8.0#remarks) that `The node must be of type 'JsonValue'` —and, when we call `JsonNode.JsonValueKind`, we may get confused:

In [34]:
jNode["top"]?["three"]?.GetValueKind()

Object

The compiler is telling us that our `three` node is _not_ of type `JsonValue` while the `JsonNode.GetValueKind` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.getvaluekind?view=net-8.0)] is dutifully returning `Object` 😐 The innocent may incorrectly assume that `Object` is but one of the many `JsonValue` “kinds” because we have made the mistake of thinking that `GetValueKind` is actually short for `GetJsonValueKind`. We will have to get a new pair of glasses 👓✨

The recursive output of a Jupyter notebook displaying our `jNode` without my `AsJupyterDisplayDictionary` extension method, will show that `three` is of type `JsonObject` while the other sibling properties are of type `JsonValuePrimitive<JsonElement>`.

The assumption here is that `GetValue<T>` only works on nodes of type `JsonValuePrimitive<T>`.

BTW: can we get `JsonElement` from one of the primitive properties?

In [35]:
jNode["top"]?["one"]?.GetValue<JsonElement>()

"this is first"

The short answer to my question is _yes_. And the long answer might go into the weeds of how to maintain “elegant” API design in one namespace, `System.Text.Json.Nodes`, while dealing with legacy types in `System.Text.Json`.

## the `JsonNode.DeepClone` method, available in .NET 8

My real intent behind the folly of `GetValue<JsonObject>()` is what Microsoft provides in the .NET 8 time-frame—the `JsonNode.DeepClone` method [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode.deepclone?view=net-8.0)]:

In [36]:
jNode["top"]?["three"]?.DeepClone().AsJupyterDisplayDictionary()

key,value
p1,"""this is three-point-one"""
p2,"""this is three-point-two"""
my-date,"""2019-11-22T05:58:34.573Z"""


Under the .NET 6 regime, we can resort to workarounds like this:

In [37]:
var json = jNode["top"]?["three"]?.ToJsonString();

JsonObject.Parse(json).AsJupyterDisplayDictionary()

key,value
p1,"""this is three-point-one"""
p2,"""this is three-point-two"""
my-date,"""2019-11-22T05:58:34.573Z"""


Let us go back to our previous caveat: ‘you cannot rename a `JsonObject` property name…’ and ‘rename’ the property name for our `JsonObject` under the tyranny of .NET 6:

In [38]:
jNode["top"]["three-changed"] = JsonObject.Parse(json);
jNode["top"].AsObject().Remove("three");

jNode.AsJupyterDisplayDictionary()

key,value
top,"{ ""one"": ""this is first"", ""two"": true, ""two-point-five"": 2.5, ""three-changed"": { ""p1"": ""this is three-point-one"", ""p2"": ""this is three-point-two"", ""my-date"": ""2019-11-22T05:58:34.573Z"" } }"


## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼